In [ ]:
# importing libraries
import os
import time
import pandas as pd
import numpy as np
import soundfile as sf
import librosa

In [ ]:
#set directory
d = 'ravdess'
df = pd.DataFrame()
temp =[]
##features from file names
df['file'] = [x for x in os.listdir(d)] #create file column of filenames
df['emotion']=[x.split('-')[2] for x in df['file']] #read emotion code from filename
#df['emotion'].replace({'01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'},inplace=True)
#for x in [x.split('-')[6][0:2] for x in df['file']]: #read actor code from filename
       # if int(x%2)==0:
       #     df['gender']='female' #even = female, odd =male, store in gender column
      #  else:
          #  df['gender']='male'
df['gender'] = [x.split('-')[6][0:2] for x in df['file']]
df['intensity']=[x.split('-')[3] for x in df['file']] #read intensity code from filename 
#df['intensity'].replace({'01':'normal','02':'strong'},inplace=True) #replace intensity code and store in intensity column
temp.append(df) #append all folder data to temp list

In [ ]:
rms = []
chroma_stft = []
spec_cent = []
spec_bw = []
spec_rolloff = []
zcr = []
mfcc = []
file_name = []

filepath = [os.path.join(d, x) for x in df['file']] #create path to individual files
for file in filepath: #iterate each file and retrieve mel_spec data
    data, sample_rate = librosa.load(file)
    file_name = file
    rms.append(np.mean(librosa.feature.rms(y=data).T, axis=0))
    chroma_stft.append(np.mean(librosa.feature.chroma_stft(y=data, sr=sample_rate).T, axis=0))
    spec_cent.append(np.mean(librosa.feature.spectral_centroid(y=data, sr=sample_rate).T, axis=0))
    spec_bw.append(np.mean(librosa.feature.spectral_bandwidth(y=data, sr=sample_rate).T, axis=0))
    spec_rolloff.append(np.mean(librosa.feature.spectral_rolloff(y=data, sr=sample_rate).T, axis=0))
    zcr.append(np.mean(librosa.feature.zero_crossing_rate(data).T, axis=0))
    mfcc.append(np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0))

df1 = pd.DataFrame()
df1['rms'] = rms
df1['chroma_stft'] = chroma_stft
df1['spec_cent'] = spec_cent
df1['spec_rolloff'] = spec_rolloff
df1['zcr'] = zcr
df1['mfcc'] = mfcc
df1['file'] = [x for x in os.listdir(d)]

In [ ]:
df = pd.merge(df, df1, how='inner', left_on='file', right_on='file')
gender_test=[]
for i in range(len(df)):
    if int(df['gender'][i]) % 2 == 0:
        gender_test.append('01')
    else:
            gender_test.append('02')
df['gender'] = gender_test
df = df.loc[(df['emotion'] == '01')|(df['emotion'] == '03')|(df['emotion'] == '04')].reset_index(drop=True)
df = df.drop('file',1)
d = [pd.DataFrame(df[col].tolist()).add_prefix(col) for col in df.columns]
df = pd.concat(d, axis=1)
df = df.rename(columns={"emotion0":"emotion","gender0":"gender","intensity0":"intensity"})
df